<img src="images/steno3d.png" width="200"></img>
<br>

<h1 style="color:#000080;"> Point Example: Airports location </h1> 

This project shows the location of 6977 airports all around the world <sup> <a href="http://openflights.org/data.html">[1]</a> </sup>. The data source is a simple comma-separated file. We can read the latitude, longitude and altitude of each airport and convert the values into points in 3D space. These points are shown on the surface of a globe which represents the Earth.

<div>
    <figure>
    <div style="height: 300px;">
    <img src="images/airport1.png" style="float: right; width: 48%; margin-top: 3%; ">
    <img src="images/airport2.png" style="float: left; width: 48%; margin-top: 3%;">
    </div>
    <figcaption align="middle"><div style="width: 50%; margin-left: auto; margin-right: auto; font-weight: bold; font-style: italic; font-size: 15px; margin-top: 20%; margin-bottom: 2%"> Figure 1: Visualization of global airport locations in Steno3D </div>
    </figcaption>
    </figure>
</div>

<h3 style="color:#000080;"> Import Dependencies </h3>

In [ ]:
import numpy as np
import steno3d

<h3 style="color:#000080;"> Login to Steno3D </h3>

Before using Steno3D, you need to <a href="https://steno3d.com/signup">sign up for an account</a> to get your own developer API key if you do not have one already. Running the cell below will provide you with instructions for how to obtain and enter your key. 

The option to `skip_credentials` prevents saving your developer key to the mybinder.org server. When working on your local computer, remove this argument and your key will be saved.

In [ ]:
steno3d.login(skip_credentials=True) 

<h3 style="color:#000080;"> Steno3D Points </h3>

Before moving to the next step, we need to know how we can draw points in Steno3D. In this section we briefly describe the process of drawing points in Steno3D before making the actual example. The way that Steno3D works is to first create a mesh and then bind the data to the mesh. The mesh defines the geometry of the pointset. So the first step to create points is to make a 0D mesh (in Steno3D, we refer to point mesh as 0D mesh). Point meshes are formed by vertices. A simple example of a point mesh is depicted in the Figure 2.  

<figure>
    <img src="images/mesh0d.png" width ="700px" height="400px" align= middle style= "margin-top: 3%">
    <figcaption align="middle"><div style="width: 30%; margin: auto; font-weight: bold; font-style: italic; font-size: 15px; margin-bottom: 5%; margin-top: 3%"> Figure 2: Point Mesh (Mesh0D)</div> </figcaption>
</figure>

As shown in this figure, vertices are 3D points represented by (x, y, z) coordinates. Vertices must always be a n x 3 array of spatial coordinates. Additional documentation for Mesh0D can be found <a href="https://python.steno3d.com/en/latest/content/api/resources/point.html#meshes">online</a>.

After making the 0D mesh, we can bind the `data` to the points. This is simply done by using a dictionary containing `location` (nodes/vertices,'N', is the only available location for points) and `data`, a <a href="https://python.steno3d.com/en/latest/content/api/resources/data.html#resources-data">DataArray</a>. The length of point data must equal the length of point mesh vertices. In this example we do not bind data to the points. 

Another feature for Steno3D points is that we can map an image to a point mesh. We refer to the images projected on a point as `textures`. Unlike data, `textures` do not correspond to certain mesh locations but instead are defined using spatial coordinates. In this example, we do not add `textures` to the points. For more information on `textures` please check <a href="https://python.steno3d.com/en/latest/content/api/resources/texture.html">Texture API documentation</a>.

<h3 style="color:#000080;"> Create a Project </h3>

Before making any resources (points, lines, surfaces or volumes) in Steno3D, we need to create a `Project` to assign the resource to that project. Projects organize all your resources together and can contain a combination of resources; they can also be viewed and shared in Steno3D. 

Following are the properties of projects that you can modify if desired:

<ul>
    <li><b>title</b>: Title of the project</li>
    <li><b>description</b>: Description of the project</li>
    <li><b>public</b>: If True, the project is viewable by anyone online. If False, the project is private and only viewable by people you choose to share it with. Free and Academic accounts have limits on the number of private projects allowed; if you would like more, upgrade your account at steno3d.com.</li>
    <li><b>resources</b>: List of resources the project contains.</li>
</ul>

You can <a href="https://steno3d.com/explore">explore public projects on steno3d.com</a>. Additional <a href="https://python.steno3d.com/en/latest/content/api/projects.html">API documentation</a> is available online.

In [ ]:
airports_proj = steno3d.Project(
    title='Airports', 
    description='Airports Locations All Around the World',
    public=False
)

<h3 style="color:#000080;"> Read Airports Data</h3> 

For this notebook, we will be using example data from the `steno3d.examples.PointExample_Airports` module. This module downloads raw data, saves it to a folder in the home directory (or another location you may specify), and allows easy access to all different pieces of data.

In [ ]:
from steno3d.examples import Airports
Airports.fetch_data(verbose=False)  # To specify a different data directory than the
                                    # default '$HOME/.steno3d_python_assets/' use
                                    # keyword argument `directory='/path/to/your/directory/'`

This function reads the Airports data and returns lists of latitudes, longitudes and altitudes. 

In [ ]:
DEG2RAD = np.pi/180
FT2KM = 12*2.54/100/1000

def read_airports_data(filename):
    """Extract latitude, longitude, and altitude from file"""
    lat = [] # Latitude
    lon = [] # Longitude
    alt = [] # Altitude
    with open(filename) as f:
        for line in f:
            data = line.rstrip().split(',')
            lat.append(float(data[6])*DEG2RAD)
            lon.append(float(data[7])*DEG2RAD)
            alt.append(float(data[8])*FT2KM)
    return np.array(lat), np.array(lon), np.array(alt)

lat, lon, alt = read_airports_data(Airports.datafile)

<h3 style="color:#000080;"> Convert latitude, longitude and altitude into points in 3D space </h3>



In [ ]:
RADIUS = 6371

def geo_to_xyz(lat, lon, alt):
    """Convert from geo coordinates to xyz
    
    Inputs:
        lat: latitude
        lon: longitude
        alt: altitude
        
    Outputs:
        x, y, z: spatial coordiantes relative to the center of the earth
        
    Note:
        This function assumes a shpherical earth
    """
    
    x = (RADIUS + alt)*np.cos(lat)*np.cos(lon)
    y = (RADIUS + alt)*np.cos(lat)*np.sin(lon)
    z = (RADIUS + alt)*np.sin(lat)
 
    return x, y, z

<h3 style="color:#000080;"> Point Mesh (Mesh0D) </h3>

In Steno3D, points use `Mesh0D` which only contains vertices. Now we can use the `geo_to_xyz` function to generate vertices for a point mesh. `Mesh0D` have properties:

<ul>
    <li><b>vertices</b> (required)<sup><b>*</b></sup>: n x 3 array of spatial coordinates  
    <li><b>title</b></li>
    <li><b>description</b></li>
</ul>

(*): For the Point resource to pass validation prior to upload, required properties must be set.

In [ ]:
my_verts = np.c_[geo_to_xyz(lat, lon, alt)]

airports_mesh=steno3d.Mesh0D(
    vertices=my_verts
)

<h3 style="color:#000080;"> Create Point </h3>

Now that we have the meshes for the pointsets, we can create the points. The following are point properties in Steno3D:

<ul>
    <li><b>project</b> (required): The project(s) that contain the resource</li>
    <li><b>mesh</b> (required): Points use `Mesh0D`, a mesh that only contains `vertices` </li>
    <li><b>data</b>: A list of `DataArrays` bound to the mesh</li>
    <li><b>textures</b>: A list of `Texture2DImages`</li>
    <li><b>opts</b>: A dictionary of options</li>
    <li><b>title</b></li>
    <li><b>description</b></li>
</ul>

Additional <a href="https://python.steno3d.com/en/latest/content/api/resources/point.html">API documentation</a> is available online. 

In [ ]:
airports_point = steno3d.Point(
    project=airports_proj,
    title='Airports Locations',
    mesh=airports_mesh,
)

<h3 style="color:#000080;"> Point Option </h3>

We can set the options for the point. These include `color` and `opacity`.

In [ ]:
airports_point.opts = dict(
    color=[34, 139, 34],
    opacity = 1
)

<h3 style="color:#000080;"> Upload and Plot </h3>

Calling `upload` will check that all aspects of surface and its containing project are built correctly, then upload the project to <a href="https://steno3d.com">steno3d.com</a>.

In [ ]:
airports_point.upload()

In [ ]:
airports_point.plot()